<a href="https://colab.research.google.com/github/sanka-eu/nonlinear_and_adaptive_control_practice/blob/main/lab1_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 1
Средства автоматизации расчетов и моделироаания систем управления в среде Python/scipy

### Модель системы

$\dot{dP}\;=\;17T\;+\;\zeta\;-\;0.1dP^3\;+\;9\sin\left(4dP\;-\;7\right)\;-\;6$

$\dot T\;=\;-0.1T^3 - 2Tcos\left(5dP\;+\;6\right)\;+\;30tanh\left(I\right)\;-\;1$

где:\
T - температура\
dP - изменение давления газа в резервуаре\
I - сила тока, протекающего через нагревательный элемент\
ζ ∈ [4;23] - некотролируемое возмущение в системе

$\dot P = - P^3 + dm + v(t)$

$\dot dm = -dm + G \, tanh(\omega)$

### 1. Автоматизация расчета закона управления с помощью пакета sympy

In [5]:
import sympy
#создание символьных переменных
dP=sympy.symbols('dP')
T=sympy.symbols('T')
zeta=sympy.symbols('zeta')
I=sympy.symbols('I')

#запись уравнений
d_dP=-P**3 + dm + v + v0
d_T=-dm + G * w

#печать сивольных выражений
print('Уравнения системы:')
print('dP/dt=' + str(dP))
print('ddm/dt=' + str(ddm))
print(str(sympy.expand(dP*ddm)))
print(str(dP*ddm))
print('')

Уравнения системы:
dP/dt=-P**3 + dm + v + v0
ddm/dt=G*w - dm
-G*P**3*w + G*dm*w + G*v*w + G*v0*w + P**3*dm - dm**2 - dm*v - dm*v0
(G*w - dm)*(-P**3 + dm + v + v0)



$\dot P_d = P_d - P$

$\dot P - \dot P_d = 0$

In [6]:
from sympy.solvers import solve
DdP=Pd-P
Ddm=solve(dP-DdP,dm)
print('Желаемая функция dm(t) для эталонной модели dP='+str(DdP))
print('dm='+str(Ddm))
print('')

Желаемая функция dm(t) для эталонной модели dP=-P + Pd
dm=[P**3 - P + Pd - v - v0]



In [ ]:
psi=Ddm[0]-dm
dpsi=sympy.diff(psi,P)*dP + sympy.diff(psi,dm)*ddm
u=solve(sympy.expand(dpsi+psi),w)
u_analytical = u[0]
print('Закон управления по методу АКАР для макропеременной psi=' + str(psi))
print('w='+str(u_analytical))
print()

### 2. Численное моделирование системы ОДУ

In [ ]:
import matplotlib.pyplot as plt
import scipy.integrate as mdl
import math
import numpy as np

u = 20.0
x0d = 3.0
def F(x, t):
    return [-math.pow(x[0],3)+x[1], -x[1]+0.25*math.tanh(-1.0*x[0])]

u = 5.0
t = np.linspace(0, 10, 10000)
y = mdl.odeint(F, [2.0, 0.0], t)

plt.plot(t,y)
plt.show()

In [ ]:
print(t)

### 3. Моделирование САУ с цифровым ПИ-регулятором


Базовый пакет для моделирования контроллера

In [ ]:
!pip install digicon_mod

In [ ]:
import digicon_mod

Правая часть дифференциального уравнения объекта со ступенчатым изменением параметра

In [ ]:
def F_with_change(step_time, init_value, finish_value):
    def F_with_control(uc):
        def F_internal(x, t):
            if t > step_time:
                v0 = finish_value
            else:
                v0 = init_value
            return [ -math.pow(x[0],3) + x[1] + v0, -x[1] + uc]

        return F_internal
    return F_with_control

Реализация алгоритма ПИ-регулирования

In [ ]:
class PI(digicon_mod.plc.PLC):
    def __init__(self, goal, Kp, Ki, gain, step):
        super(PI,self).__init__(gain, step)
        self.Ki = Ki
        self.Kp = Kp
        self.goal = goal
        self.ei = 0

    def control(self, x, t):
        e = x[0] - self.goal
        self.ei = self.ei + e
        return self.Kp * e + self.Ki * self.ei

Вывод результатов моделирования

In [ ]:
def plot_result(time, time_end, x1, x2, plc, goal):
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.grid()
    plt.xlim(0, time_end)
    plt.plot(time,x1, 'r-', time, x2, 'b-')
    plt.plot([0, time_end], [goal, goal], color='#FF0000',linestyle='--')
    plt.subplot(1,2,2)
    plt.grid()
    plt.xlim(0, time_end)
    plt.plot(plc.t, plc.u,'b-',plc.t,plc.u_lim,'r-')
    plt.plot([0, tk], [1, 1], 'r--',[0, tk], [-1, -1], 'r--',[0, tk],[0, 0],'r:')
    plt.show()

Программа моделирования

In [ ]:
v0_init = 0.0
v0_finish = 1.0
change_time = 15
goal = 0.55
gain = 3.0
step = 0.5
mod_step = 0.1
func_ctrl = F_with_change(change_time,v0_init,v0_finish)

plc=PI(goal=goal, Kp=-1.0, Ki=-0.05, gain=gain, step=step)
tk=30
x0=[-2.5, 0.1]
res = digicon_mod.sim.calculate(func_ctrl, x0, mod_step, tk, plc)
plot_result(time=res['t'], time_end=tk, x1=res['x1'], x2=res['x2'], plc=plc, goal=goal)